# Load Packages

In [15]:

%load_ext autoreload
%autoreload 2

import sys
from os.path import join
from tqdm.auto import tqdm
import joblib
import torch
sys.path.append("../../")

from src.file_manager.load_data import load_split_dict
from src.file_processing.processing_predictions import load_prediction_df_dict, save_pi_df_dict
from src.file_processing.processing_predictions import load_pi_df_dict
from src.file_manager.save_load_scaler import load_scaler
from src.pi_methods.knn import knn_prediction_interval
from src.pi_methods.weighted import weighted_prediction_interval
from src.pi_methods.cond_gaussian import cond_gauss_prediction_interval
from src.pi_methods.gaussian_copula import gauss_copula_prediction_interval
from src.pi_methods.conformal_prediction import conformal_prediction_interval
from src.evaluation.investigate_re_pe_distribution import investigate_re_n_pe_for_all_seeds, investigate_re_n_pe
from src.misc import create_folder
from ue_pi_dicts import ue_dict
# from seed_file import seed

seed_list = [2023, 2024, 2025, 2026, 2027]
data_label = "physionet"
batch_size = 64

# File paths
fp_notebooks_folder = "../"
fp_project_folder = join(fp_notebooks_folder, "../")
fp_data_folder = join(fp_project_folder, "../", "data")
fp_output_data_folder = join(fp_data_folder, data_label)
fp_checkpoint_folder = join(fp_project_folder, "checkpoints")
fp_project_checkpoints = join(fp_checkpoint_folder, data_label)
fp_tuning = join(fp_project_checkpoints, "tuning")
fp_models = join(fp_project_checkpoints, "models")
fp_predictions = join(fp_project_checkpoints, "predictions")
fp_pi_predictions = join(fp_project_checkpoints, "pi_predictions")

pred_file_names =  ["rue", "gpr", "infernoise", "der", "bnn"]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Load Data

In [ ]:
split_dict = load_split_dict(fp_output_data_folder)
scaler = load_scaler(join(fp_output_data_folder, "scaler.pickle"))

# Investigate Recon-Pred Error Space

In [17]:
investigate_re_n_pe(pred_df_dict, split_dict)

  0%|          | 0/3 [00:00<?, ?it/s]

/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))


,dist\ntest_df,dist\nvalid_df,dist meaning\ntest_df,dist meaning\nvalid_df,marginal gauss\ntest_df,marginal gauss\nvalid_df,normality pval test_df,normality pval valid_df,normality stat test_df,normality stat valid_df
t+1,0.194552,0.214747,0.084094,0.091607,0.0,0.0,0.0,0.0,5.000152,1.608526
t+2,0.102844,0.112172,0.065991,0.081030,0.0,0.0,0.0,0.0,5.032879,1.714476
t+3,0.142527,0.156089,0.060066,0.075562,0.0,0.0,0.0,0.0,4.873604,1.614015


In [18]:
investigate_re_n_pe_for_all_seeds(
    seed_list=seed_list, split_dict=split_dict, fp_pi_predictions=fp_pi_predictions)

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

Loaded df_dict!


  0%|          | 0/3 [00:00<?, ?it/s]

/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))


...

  0%|          | 0/3 [00:00<?, ?it/s]

Loaded df_dict!


  0%|          | 0/3 [00:00<?, ?it/s]

/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))
/home/lirong/miniconda3/envs/main/lib/python3.10/site-packages/scipy/stats/_stats_py.py:9694: RuntimeWarning: divide by zero encountered in log
  statistic = -2 * np.sum(np.log(pvalues))


,dist\ntest_df,dist\nvalid_df,dist meaning\ntest_df,dist meaning\nvalid_df,marginal gauss\ntest_df,marginal gauss\nvalid_df,normality pval test_df,normality pval valid_df,normality stat test_df,normality stat valid_df
t+1,0.17143 ± 0.022,0.1868 ± 0.025,0.08346 ± 0.003,0.09566 ± 0.003,0.0 ± 0.000,0.0 ± 0.000,0.0 ± 0.000,0.0 ± 0.000,4.40893 ± 0.604,1.54139 ± 0.056
t+2,0.12924 ± 0.016,0.13988 ± 0.017,0.06957 ± 0.002,0.07986 ± 0.002,0.0 ± 0.000,0.0 ± 0.000,0.0 ± 0.000,0.0 ± 0.000,4.04644 ± 0.679,1.51848 ± 0.111
t+3,0.14986 ± 0.008,0.16193 ± 0.008,0.06248 ± 0.002,0.07869 ± 0.004,0.0 ± 0.000,0.0 ± 0.000,0.0 ± 0.000,0.0 ± 0.000,3.89029 ± 0.704,1.46618 ± 0.103
